In [ ]:
l0gins = [
    {'wpDomain':'yourdomain.com', 
     'wpUN':'admin', #The user needs to have admin privs
     'wpPW':'slkdfjsldfjsldkjf', #Wordpress Application Password of an account with admin privs
     'post_status':'draft', #or 'publish'
     'kw':['pubic hairloss', 'greying armpit hair'],
     'pexels_key':'lsdfjsdlfjslkdjflkdjf',
     'openai_key':'skldfjslkdfjslkdfjjdf'
     }
    ]

In [ ]:
!nvidia-smi
!pip install torch transformers sentencepiece protobuf==3.20 openai

In [ ]:
from io import BytesIO as bIO
import openai, torch
from json import loads as jLD, dumps as jD
from requests import get as g, post as p
from datetime import datetime as dT, timedelta as tD
from PIL import Image as img, ImageEnhance as iE, ImageFilter as iF
from secrets import token_hex as heX
from base64 import b64encode as be
from os import  path as pt, remove as rm
from sys import exit as eX
from time import perf_counter as pC, sleep as zZz
from random import uniform as u, choice as c, randint as ri, randrange as rr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
aiStart = pC()
print('Instantiating the AI')
model = AutoModelForSeq2SeqLM.from_pretrained("ramsrigouthamg/t5_sentence_paraphraser")
tokenizer = AutoTokenizer.from_pretrained("ramsrigouthamg/t5_sentence_paraphraser")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print('AI instantiated in', pC()-aiStart)
beams = sequences = 15

In [ ]:
def deDupeQ(my_list):
  jpop=[]
  for i in range(len(my_list)):
    for j in range(i+1, len(my_list)):
      if my_list[i] == my_list[j]:
        jpop.append(j)
  print(f'{len(jpop)} dupes found in qList')
  for j in reversed(sorted(jpop)):
    my_list.pop(j)
  return my_list

def imgJSON(kw, qLen):
  totalimg = qLen * 4
  imJson = g("https://api.pexels.com/v1/search", params={"query": kw,"per_page": 80,"page": 1}, headers={"Authorization": f"Bearer {pexels_api_key}"}).json()
  for page in range(round(totalimg/80)-1):
    zZz(1)
    imJson['photos'].extend(g("https://api.pexels.com/v1/search", params={"query": kw,"per_page": 80,"page": page+2}, headers={"Authorization": f"Bearer {pexels_api_key}"}).json()['photos'])
  dedup = deDupeQ(imJson['photos'])
  imJson['photos'] = dedup
  print(f"Got {len(imJson['photos'])} images from pexels that had {len(imJson['photos'])-len(dedup)} dupes effectively leaving {len(dedup)} available for an estimated requirement of {totalimg} for this run")
  return imJson

def randomDateTime(n, days=30, repeats=10, hours_diff=3):
    # Define start and end dates
    start_date = dT.now()
    end_date = start_date + tD(days=days)

    # Calculate time delta and maximum number of repetitions per date
    delta_seconds = (end_date - start_date).total_seconds() / n
    max_repeats = int(n / (days * repeats))

    # Generate timestamps
    timestamps = set()
    current_date = start_date
    count = 0
    while len(timestamps) < n:
        for i in range(repeats):
            # Check if the maximum number of repetitions for the current date has been reached
            if count >= max_repeats:
                current_date += tD(days=1)
                count = 0
            # Generate a timestamp for the current date with a random time
            rand_seconds = u(0, delta_seconds)
            date = current_date + tD(seconds=rand_seconds)
            timestamp = date.strftime("%Y-%m-%dT%H:%M:%S")
            while timestamp in timestamps:
                # Regenerate the timestamp if it already exists in the set
                rand_seconds = u(0, delta_seconds)
                date = current_date + tD(seconds=rand_seconds)
                timestamp = date.strftime("%Y-%m-%dT%H:%M:%S")
            timestamps.add(timestamp)
            count += 1
        # Move to the next date
        current_date += tD(hours=hours_diff)

    # Return the list of timestamps
    return list(timestamps)

def openAI(text):
  retry = 0
  t = 10
  while True:
    try:
      return openai.Completion.create(model="text-davinci-003", prompt=text, max_tokens=1000, temperature=0.7)['choices'][0]['text']  
    except Exception as e:
      retry +=1
      print(f'OpenAI threw an error {e}, sleeping for {t} seconds. Retry {retry} of 5')
      if retry == 5:
        eX(f'Looks like openAI is fucked. See you on the otherside')
      else:
        zZz(t)
        t += 10
        continue

def contentProcessor(text):
    def paraPhraser(input_text):
        text = "paraphrase: "+input_text + " </s>"
        encoding = tokenizer.encode_plus(text,max_length =256, padding='max_length', return_tensors="pt")
        input_ids,attention_mask  = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
        model.eval()
        beam_outputs = model.generate(
            input_ids=input_ids,attention_mask=attention_mask,
            max_length=256,
            early_stopping=True,
            num_beams=beams,
            num_return_sequences=sequences
        )
        sendBack = []
        for beam_output in beam_outputs:
            sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
            sendBack.append(str(sent).strip('paraphrasedoutput: '))
        return sendBack
    finalText = ''
    print(f"Processing \n\n{text}\n\nof {len(text.split())} words in text\n\n\n")
    cPTimer = pC()
    paraSplit = list(filter(lambda x: x != '', text.split('\n\n')))
    paraLength = len(paraSplit)
    for para in paraSplit:
        phrases = para.split('. ')
        phraseLength = len(phrases)
        for phrase in phrases:
            rphResponse = paraPhraser(input_text=phrase)
            finalText += c(rphResponse)
            if phraseLength > 1:
                finalText += ' '
        if paraLength > 1:
            finalText += '\n\n'
    print(f"Processed Text \n\n{finalText}\n\n of {len(finalText.split())} words processed in {round(pC()-cPTimer)} seconds\n\n\n")
    return finalText

def imgAdder(text, dicT, imJSON):
  def imgPost(data):
    headers = {"Content-Type": "image/jpeg", "Accept": "application/json", 'Content-Disposition': f"attachment; filename={heX(20)}.jpg",}
    auth = (dicT['wpUN'], dicT['wpPW'])
    return p(url = f'https://{dicT["wpDomain"]}/wp-json/wp/v2/media', auth=auth, headers=headers, data=data).json()  
  def imgFuckery(data):
    image = img.open(bIO(data))    
    enhancer = iE.Color(image)
    fI = enhancer.enhance(u(.6,.9))
    fI = fI.filter(iF.BoxBlur(radius=u(0.5,2.5)))
    fI = fI.filter(iF.SMOOTH)
    fI = fI.filter(iF.EDGE_ENHANCE)
    fI = fI.filter(iF.SHARPEN)
    fI = fI.filter(iF.DETAIL)
    if c([0,1]) == 1:
      fI = fI.rotate(u(-.7,.7))
    if c([0,1]) == 1:
      fI = fI.transpose(img.FLIP_LEFT_RIGHT)
    imgData = bIO()
    fI.save(imgData, format='JPEG')
    return imgData.getvalue()
  def imgFetch():   
    imgList = []
    randomimgn = []
    for i in range(imgN):
      while True:
        rn = ri(0, len(imJSON['photos'])-1)
        if rn not in randomimgn:
          randomimgn.append(rn)
          break
    aDDed = []
    for _ in range(len(randomimgn)):
      while True:
        i = ri(0, len(imJSON['photos'])-1)
        if i in aDDed:
          continue
        else:
          aDDed.append(i)
          if 'large' in imJSON['photos'][i]['src']:
            imgSize = 'large'
            break
          elif 'medium' in imJSON['photos'][i]['src']:
            imgSize = 'medium'
            break
          elif 'small' in imJSON['photos'][i]['src']:
            imgSize = 'small'
            break
          else:
            continue      
      imgList.append(imgPost(data=imgFuckery(g(imJSON['photos'][i]['src'][imgSize]).content))['guid']['raw'])
      print(f"Making list of images. Length of imglist now is {len(imgList)}")
    print('And BOOM! thats done')
    return imgList
  paras = list(filter(lambda x: x != '', text.split('\n\n')))
  paralen = len(paras)
  print('paralength',paralen)
  imgN = ri(2, round(paralen/2)) if paralen > 3 else 1
  print('imgn',imgN)
  imgInterval = []
  if imgN == 1 and paralen == 2:
    imgInterval.append(1)
  elif imgN == 1 and paralen == 1:
    imgInterval.append(0)
  else:
    for i in range(imgN):
      reTRy = 0
      while True: 
        rnrint = ri(0,paralen-2)
        if rnrint not in imgInterval:
          imgInterval.append(rnrint)
          break
        reTRy += 1
        if reTRy > 100:
          break
  print('imgInterval',imgInterval)
  imglist = imgFetch()
  print('imglist length',len(imglist))
  imglistn = 0
  article = ''
  for i, para in enumerate(paras):
    article += f'{para}\n\n'
    if i in imgInterval and imglistn <= len(imglist)-1:
        if i==0:
          article = f'<img src="{imglist[imglistn]}" /> \n\n  {article}'
        else:
          article += f'<img src="{imglist[imglistn]}" /> \n\n'
        imglistn += 1
  return article  

def qListWrite(jSON):
  with open('/content/qlist.json','w') as qListWriter:
    qListWriter.write(jD(jSON))

def qListReader():
  if not pt.exists('/content/qlist.json'):
    qListWrite({})
    qListJson = {}
  else:
    try:
      with open('/content/qlist.json','r') as qListReader:
        qListJson = jLD(qListReader.read())
    except:
      qListWrite({})
      qListJson = {}
    else:
      return qListJson

  return qListJson

def qListMaker(kw):
  topicsList = openAI(f"give me a list topics around {kw} as a comma separated list without any serial numbers").replace("\n",'').split(',')
  for i, t in enumerate(topicsList):
    print(f'{i+1}. {t.strip()} in {kw}')
  qList = []
  for topic in topicsList:
    print('Pulling questions for', topic)
    zZz(2)
    qList.extend(openAI(f"a list questions around the topic {f'{topic.strip()} in {kw}'} as a comma separated list without serial numbers").replace("\n",'').split(','))
  qLIST = []
  qlen = len(qList)
  conversionsTart = pC()
  for i, q in enumerate(qList):
    q = ''.join([s.strip('\"') for s in q]).strip()
    print(f"Converting {i} of {qlen}, Title: {q.strip()} to a blog title. Time elapsed : {pC()-conversionsTart}")
    qLIST.append(''.join([s.strip('\"') for s in contentProcessor(openAI(f'''convert this question {q} to a blog post title''').replace("\n",'').strip())]))
    print(f"Original question: {q}\nProcessed Output{qLIST[i]}\n\n")
  qList = qListJson[kw]=deDupeQ(qLIST)
  qListWrite(jSON=qListJson)
  for i, q in enumerate(qList):
    print(i+1, q)
  qListJson[kw]=deDupeQ(qList)
  qListWrite(jSON=qListJson)
  return qList

In [ ]:
for i, dicT in enumerate(l0gins):
  openai.api_key = dicT['openai_key']
  pexels_api_key = dicT['pexels_key']
  print('\n\nProceeding with', dicT['wpDomain'])
  wpHeaders = {'Authorization':'Basic '+be(f"{dicT['wpUN']}:{dicT['wpPW']}".encode()).decode()}
  wpPostsURL = f"https://{dicT['wpDomain']}/wp-json/wp/v2/posts"
  wpStatus = dicT['post_status']
  for j, kw in enumerate(dicT['kw']):
    qListJson = qListReader()
    if kw in qListJson:
      print('Found kw in qlist.json')
      qList = deDupeQ(qListJson[kw])
      if len(qList) == 0:
        print('qList is as empty as your fucking brain, pulling qs from openai')
        qList = qListMaker(kw)
      print(f'Got back {len(qList)} questions')
    else:
      print(f'Kw not found in qlist.json, this probably means this is the first time you are running this keyword. pulling questions from openai')
      qList = qListMaker(kw=kw)
      print(f'Got back {len(qList)} questions')
    qListOriginalLength = len(qList)
    sTart = pC()
    imJSON = imgJSON(kw=kw, qLen= qListOriginalLength)
    datelist = randomDateTime(n=qListOriginalLength)
    while True:
      if len(qList) == 0:
        print(f"{qListOriginalLength} posts done in {(pC()-sTart)/60} minutes")
        del qListJson[kw]
        qListWrite(jSON=qListJson)
        break
      title = c(qList)
      print(f"\n\n\nFor title '{title}', IMJson length is {len(imJSON['photos'])}, {qListOriginalLength - len(qList)+1} of {qListOriginalLength}. Time Elapsed: {round(pC()-sTart)/60} minutes")
      postStart = pC()
      print(f"{title} Posted to to {dicT['wpDomain']}\n{qListOriginalLength - (len(qList)-1)} of {qListOriginalLength} questions Posted.\n{j+1} of {len(dicT['kw'])} keywords\n{len(l0gins)-(i+1)} of {len(l0gins)} domains done. \nPost response {p(url=wpPostsURL, headers=wpHeaders, json={'title':title,'status':wpStatus, 'date_gmt':datelist.pop(rr(len(datelist))), 'content':imgAdder(text=contentProcessor(openAI(f'a detailed essay on {title}')), dicT=dicT, imJSON=imJSON)})}\nDone in {postStart-pC()} seconds\nRemaining {len(qList)-1}")
      qList.remove(title)
      qListJson[kw]=qList
      qListWrite(jSON=qListJson)
      print('Next one in 3 secs')
      zZz(3)